In [1]:
import deepcut
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
from pythainlp import word_vector
import re,string
from pythainlp import word_vector
from numpy import argmax
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import random
from keras_contrib.layers import CRF

In [189]:
def clean_msg(msg):
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<*?>','', msg)
    msg = re.sub(r'\d+', '',msg)
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    msg = re.sub(r'-','',msg)
    msg = re.sub(r'ฯ','',msg)
    msg = re.sub(r'ๆ','',msg)
    msg = re.sub(r'!@#$','',msg)
    msg = re.sub(r'[a-zA-Z]','',msg)
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [190]:
input_txt = open("corpus1.txt","r")
text = input_txt.read()
txt = clean_msg(text)
input_txt.close()
corpus = txt.split()
print(corpus)

['พยากรณ์อากาศ', 'ชั่วโมงข้างหน้า', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังปานกลางจากประเทศจีนแผ่ปกคลุมประเทศไทยตอนบน', 'ประกอบกับมีหย่อมความกดอากาศต่ำปกคลุมบริเวณประเทศกัมพูชา', 'ลักษณะเช่นนี้ทำให้ประเทศไทยตอนบนมีฝนเกิดขึ้น', 'กับมีอากาศเย็น', 'ส่วนบริเวณยอดดอยและยอดภู', 'มีอากาศหนาว', 'สำหรับมรสุมตะวันออกเฉียงเหนือพัดปกคลุมอ่าวไทยและภาคใต้', 'ทำให้ภาคใต้มีฝนตกต่อเนื่อง', 'อนึ่ง', 'พายุโซนร้อน', 'อัสนี', 'พายุระดับ', 'บริเวณตะวันตกเฉียงใต้ของเกาะไต้หวัน', 'มีแนวโน้มที่จะอ่อนกำลังลง', 'และไม่มีผลกระทบต่อประเทศไทย', 'ในช่วงวันที่', 'พย', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังปานกลางจากประเทศจีนปกคลุมประเทศไทยตอนบน', 'ประกอบกับมีหย่อมความกดอากาศต่ำปกคลุมบริเวณประเทศกัมพูชา', 'ลักษณะเช่นนี้ทำให้บริเวณประเทศไทยตอนบนมีอากาศเย็นกับมีฝนเล็กน้อยถึงปานกลาง', 'โดยเฉพาะบริเวณภาคตะวันออกเฉียงเหนือตอนล่างและภาคตะวันออก', 'สำหรับภาคใต้ตอนล่างมีฝนตกหนักบางแห่ง', 'เนื่องจากมรสุมตะวันออกเฉียงเหนือพัดปกคลุมอ่าวไทยและภาคใต้', 'ส่วนในช่วงวันที่', 'พย', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังค่อน

In [191]:
def word_tokenize(corpus):
    x = 0
    words = []
    for i in corpus:
        list_word = deepcut.tokenize(i,custom_dict='custom_dict.txt') #ตัดคำ
        words.append(list_word)
        x += 1
    return words
# corpus=word_tokenize(corpus)
# x_test = word_tokenize(x_test)
words = word_tokenize(corpus)
print(words)
# print(len(words))

[['พยากรณ์อากาศ'], ['ชั่วโมง', 'ข้าง', 'หน้า'], ['บริเวณ', 'ความ', 'กด', 'อากาศ', 'สูง', 'หรือ', 'มวล', 'อากาศ', 'เย็น', 'กำลัง', 'ปานกลาง', 'จาก', 'ประเทศ', 'จีน', 'แผ่', 'ปกคลุม', 'ประเทศ', 'ไทย', 'ตอน', 'บน'], ['ประกอบ', 'กับ', 'มี', 'หย่อม', 'ความ', 'กด', 'อากาศ', 'ต่ำ', 'ปกคลุม', 'บริเวณ', 'ประเทศ', 'กัมพูชา'], ['ลักษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['กับ', 'มี', 'อากาศ', 'เย็น'], ['ส่วน', 'บริเวณ', 'ยอด', 'ดอย', 'และ', 'ยอดภู'], ['มี', 'อากาศ', 'หนาว'], ['สำหรับ', 'มรสุม', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ', 'พัด', 'ปกคลุม', 'อ่าวไทย', 'และ', 'ภาค', 'ใต้'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่อง'], ['อนึ่ง'], ['พายุ', 'โซนร้อน'], ['อัสนี'], ['พายุ', 'ระดับ'], ['บริเวณ', 'ตะวัน', 'ตก', 'เฉียง', 'ใต้', 'ของ', 'เกาะ', 'ไต้หวัน'], ['มี', 'แนวโน้ม', 'ที่', 'จะ', 'อ่อน', 'กำลัง', 'ลง'], ['และ', 'ไม่', 'มี', 'ผล', 'กระทบ', 'ต่อ', 'ประเทศ', 'ไทย'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['พย'], ['บริเวณ', 'ความ', 'กด', 'อา

In [192]:
print(words)

[['พยากรณ์อากาศ'], ['ชั่วโมง', 'ข้าง', 'หน้า'], ['บริเวณ', 'ความ', 'กด', 'อากาศ', 'สูง', 'หรือ', 'มวล', 'อากาศ', 'เย็น', 'กำลัง', 'ปานกลาง', 'จาก', 'ประเทศ', 'จีน', 'แผ่', 'ปกคลุม', 'ประเทศ', 'ไทย', 'ตอน', 'บน'], ['ประกอบ', 'กับ', 'มี', 'หย่อม', 'ความ', 'กด', 'อากาศ', 'ต่ำ', 'ปกคลุม', 'บริเวณ', 'ประเทศ', 'กัมพูชา'], ['ลักษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['กับ', 'มี', 'อากาศ', 'เย็น'], ['ส่วน', 'บริเวณ', 'ยอด', 'ดอย', 'และ', 'ยอดภู'], ['มี', 'อากาศ', 'หนาว'], ['สำหรับ', 'มรสุม', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ', 'พัด', 'ปกคลุม', 'อ่าวไทย', 'และ', 'ภาค', 'ใต้'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่อง'], ['อนึ่ง'], ['พายุ', 'โซนร้อน'], ['อัสนี'], ['พายุ', 'ระดับ'], ['บริเวณ', 'ตะวัน', 'ตก', 'เฉียง', 'ใต้', 'ของ', 'เกาะ', 'ไต้หวัน'], ['มี', 'แนวโน้ม', 'ที่', 'จะ', 'อ่อน', 'กำลัง', 'ลง'], ['และ', 'ไม่', 'มี', 'ผล', 'กระทบ', 'ต่อ', 'ประเทศ', 'ไทย'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['พย'], ['บริเวณ', 'ความ', 'กด', 'อา

In [6]:
thai_letters = 'กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤฤๅลฦฦๅวศษสหฬอฮะัาำิีึืุูเแโใไ็่้๊๋์'

In [7]:
def ran_incorect(w):
    incorect = []
    for i in w:
        in_cor = []
        for r in i:
            if len(r) >= 5:
                x = random.randint(1, len(r)-1)
                y = random.randint(0, len(thai_letters)-1)
                x1 = []
                for ran in r:
                    x1.append(ran)
                x1[x] = thai_letters[y]
                j = ''.join(x1)
                in_cor.append(j)
            else:
                in_cor.append(r)
        incorect.append(in_cor)
    return incorect
#                 print(r[1])
#             print(len(r))

In [8]:
random.seed(1)
incorect_word = ran_incorect(words)
print(incorect_word)
print(len(incorect_word))

[['พยาฉรณ์อากาศ'], ['ชั่ฐโมง', 'ข้าง', 'หน้า'], ['บริเืณ', 'ความ', 'กด', 'อากาอ', 'สูง', 'หรือ', 'มวล', 'อาญาศ', 'เย็น', 'กำลัค', 'ปานกีาง', 'จาก', 'ประเทก', 'จีน', 'แผ่', 'ปกคลรม', 'ปรฎเทศ', 'ไทย', 'ตอน', 'บน'], ['ประคอบ', 'กับ', 'มี', 'หค่อม', 'ความ', 'กด', 'ออกาศ', 'ต่ำ', 'ปกิลุม', 'บ้ิเวณ', 'ปรึเทศ', 'กัมพ์ชา'], ['ลัษษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ปรฝเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['กับ', 'มี', 'อากาๅ', 'เย็น'], ['ส่วน', 'บำิเวณ', 'ยอด', 'ดอย', 'และ', 'ยธดภู'], ['มี', 'อากฐศ', 'หนาว'], ['สำหไับ', 'มรสุไ', 'ตะลัน', 'ออก', 'เฉีใง', 'เหนืฅ', 'พัด', 'ปกคลภม', 'อ่าวไทั', 'และ', 'ภาค', 'ใต้'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่ทง'], ['อนึ์ง'], ['พายุ', 'โซนร้อฬ'], ['อึสนี'], ['พายุ', 'รตดับ'], ['บริเวะ', 'ตะวโน', 'ตก', 'เเียง', 'ใต้', 'ของ', 'เกาะ', 'ไฦ้หวัน'], ['มี', 'แนวโน้ะ', 'ที่', 'จะ', 'อ่อน', 'กำถัง', 'ลง'], ['และ', 'ไม่', 'มี', 'ผล', 'กรขทบ', 'ต่อ', 'ปร๋เทศ', 'ไทย'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['พย'], ['บริเวพ', 'ความ', 'กด', 'อา

In [9]:
def create_tar(word,ch):
    tar_in = []
    for i in word:
        targ = []
        for j in i:
            if ch:
                if len(j) >= 5:
                    targ.append('incorrect')
                else:
                    targ.append('correct')
            else:
                targ.append('correct')
        tar_in.append(targ)
    return tar_in
# print(tar_in)

In [10]:
target_c = create_tar(words,False)
target_ic = create_tar(incorect_word,True)

In [11]:
target_c.extend(target_ic)
words.extend(incorect_word)


In [12]:
w = list(range(10))
w2 = list(range(10))

for i in words:
    x = random.randint(0, len(words)-1)
    y = random.randint(0, len(words)-1)
    t = words[x]
    t1 = words[y]
    s = target_c[x]
    s2 = target_c[y]
    words[x] = t1
    words[y] = t
    target_c[x] = s2
    target_c[y] = s

In [13]:
print(words)

[['เมตร'], ['โดย', 'มี', 'คลุ่น', 'สูง', 'ปร็มาณ'], ['โดย', 'จะ', 'มี', 'กำลัง', 'ค่อนข้าง', 'แรง', 'และ', 'ต่อเนื่อง'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่อง'], ['ลักษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['บริฦวณ', 'เ็ือกเขา'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['ลักษณะ', 'ดัง', 'กล่าว', 'ทำ', 'ให้', 'บริเวณ', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'อากาศ', 'เย็น', 'เกือบ', 'ทั่วไป', 'ใน', 'ภาค', 'เหนือ', 'และ', 'ภาค', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ'], ['ประกอบ', 'กับ', 'ใน', 'วัน', 'แรก', 'ของ', 'ช่วง', 'มี', 'หย่อม', 'ความ', 'กด', 'อากาศ', 'ต่ำ', 'ปกคลุม', 'อยู่', 'บริเวณ', 'ภาค', 'เหนือ', 'และ', 'ภาค', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ'], ['บอิเวณ', 'ที่', 'มี', 'ฝนษ้า', 'คปนอง', 'คลื่น', 'สูง', 'มาก', 'กว่า'], ['ส่วน', 'บริเวณ', 'ยอด', 'ดอย', 'และ', 'ยอดภู'], ['ส่วน', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตลอด', 'ช่วง', 'จาก', 'อิทธฌพล', 'ของ', 'มรสพม', 'ตะวัฦ', 'ออก', 'เๅียง', 'เหฦือ', 'ที่', 'พัด', 'ปกคลุล', 'อ่ววไทย', 'และ'

In [14]:
print(target_c)

[['correct'], ['correct', 'correct', 'incorrect', 'correct', 'incorrect'], ['correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct'], ['correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct'], ['correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct'], ['incorrect', 'incorrect'], ['correct', 'correct', 'correct', 'correct'], ['correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct'], ['correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'correct', 'corre

In [15]:
print(len(words))

174


In [16]:
w2v = word_vector.get_model()

In [17]:
vecter = []
vocab_ = []
index = 0
for word in w2v.index2word:
    vocab_.append(word)
    vecter.append(w2v[word])

In [18]:
print(len(target_c))
print(len(words))
n_word = len(vocab_ )

174
174


In [157]:
check_idx = {}
# check_idx['pad'] = 0
check_idx['correct'] = 0
check_idx['incorrect'] = 1

In [158]:
def sequence_word(input_text):
    idxs = list()
    for word in input_text:
        if word in vocab_:
            idxs.append(vocab_.index(word))
        else:
            vocab_.append(word)
            vecter.append(np.random.rand(*vecter[0].shape))
            idxs.append(vocab_.index(word))
    return idxs

def sequence_target(input_label):
    out = []
    for i in input_label:
        idxs = [check_idx[w] for w in i]
        out.append(idxs)
    return out

In [159]:
max_len = 30
n_check = len(check_idx)

In [160]:
X_word_train = [sequence_word(s) for s in words]
X_word_train = sequence.pad_sequences(X_word_train,maxlen=max_len,padding='post')

In [161]:
Y_train = sequence_target(target_c)
Y_train = sequence.pad_sequences(Y_train,maxlen=max_len,padding='post')
Y_train = np.array(Y_train)
print(Y_train.shape)

(174, 30)


In [162]:
X_word_train = np.array(X_word_train)
print(X_word_train.shape)

(174, 30)


In [163]:
print(X_word_train[0])

[587   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [164]:
# vocab_ = [''] + vocab_
# vecter = np.vstack((np.random.rand(*vecter[0].shape), vecter))

In [165]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [166]:
cat_train_tags_y = to_categorical(Y_train, len(check_idx))
print(cat_train_tags_y[0])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [167]:
from keras.layers import Activation

In [178]:
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=(max_len,)))
model.add(keras.layers.Embedding(vecter.shape[0], vecter.shape[1], name='embed'))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(len(check_idx))))
model.add(Activation('softmax'))

In [179]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=1e-4),metrics=['accuracy'])

In [180]:
model.get_layer('embed').set_weights([vecter])
model.get_layer('embed').trainable = True

In [181]:
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 30, 300)           15482700  
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 30, 512)           1140736   
_________________________________________________________________
time_distributed_23 (TimeDis (None, 30, 2)             1026      
_________________________________________________________________
activation_11 (Activation)   (None, 30, 2)             0         
Total params: 16,624,462
Trainable params: 16,624,462
Non-trainable params: 0
_________________________________________________________________
None


In [182]:
history = model.fit(X_word_train,np.array(Y_train),epochs=10,validation_split=0.3)

Epoch 1/10
4/4 [==============================] - 1s 329ms/step - loss: 0.2908 - accuracy: 0.9397 - val_loss: 0.2102 - val_accuracy: 0.9459
Epoch 2/10
4/4 [==============================] - 1s 142ms/step - loss: 0.1978 - accuracy: 0.9554 - val_loss: 0.2005 - val_accuracy: 0.9459
Epoch 3/10
4/4 [==============================] - 1s 141ms/step - loss: 0.1852 - accuracy: 0.9554 - val_loss: 0.1966 - val_accuracy: 0.9459
Epoch 4/10
4/4 [==============================] - 1s 142ms/step - loss: 0.1747 - accuracy: 0.9554 - val_loss: 0.1836 - val_accuracy: 0.9459
Epoch 5/10
4/4 [==============================] - 1s 142ms/step - loss: 0.1594 - accuracy: 0.9554 - val_loss: 0.1655 - val_accuracy: 0.9459
Epoch 6/10
4/4 [==============================] - 1s 141ms/step - loss: 0.1425 - accuracy: 0.9554 - val_loss: 0.1439 - val_accuracy: 0.9459
Epoch 7/10
4/4 [==============================] - 1s 141ms/step - loss: 0.1252 - accuracy: 0.9554 - val_loss: 0.1256 - val_accuracy: 0.9459
Epoch 8/10
4/4 [====

In [183]:
# scores = model.evaluate(X_word_train,to_categorical(Y_train, len(check_idx)))
# print(f"{model.metrics_names[1]}: {scores[1] * 100}")

In [184]:
x = model.predict(X_word_train[165]).argmax(axis=1)
# print(x,x.shape)

In [185]:
print(logits_to_tokens(x, {i: t for t, i in check_idx.items()}))

[['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct'], ['correct', 'correct']]
